In [ ]:
import scipy.io as sio, numpy as np, scipy
import os, re, random,concurrent.futures, time, copy
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import utils
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import models

cudnn.benchmark = True

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [ ]:
folder_path = {"Long_words": "/home/tusharsingh/DATAs/speech_EEG/Long_words",
        "Short_Long_words": "/home/tusharsingh/DATAs/speech_EEG/Short_Long_words",
        "Short_words": "/home/tusharsingh/DATAs/speech_EEG/Short_words",
        "Vowels": "/home/tusharsingh/DATAs/speech_EEG/Vowels"}

Labels = {"Long_words": {1: "cooperate",2: "independent"},
        "Short_Long_words": {1:"cooperate",2:"in"},
        "Short_words": {1:"out",2:"in",3:"up"},
        "Vowels": {1:"a",2:"i",3:"u"}}

words = {"Long_words": {"cooperate" : 0,"independent" : 1},
        "Short_Long_words": {"cooperate" : 0,"in" : 1},
        "Short_words": {"out" : 0,"in" : 1,"up" : 2},
        "Vowels": {"a" : 0, "i": 1, "u": 2}}

eeg_image_folder_path = {"Long_words": "/home/tusharsingh/DATAs/speech_EEG/EEG_image/Long_words",
        "Short_Long_words": "/home/tusharsingh/DATAs/speech_EEG/EEG_image/Short_Long_words",
        "Short_words": "/home/tusharsingh/DATAs/speech_EEG/EEG_image/Short_words",
        "Vowels": "/home/tusharsingh/DATAs/speech_EEG/EEG_image/Vowels"}

In [ ]:
# loads EEGs from the path given 
def load_EEG(path, words):
    eeg = []
    labels = []
    patient_id = []
    name = []
    for subject in os.scandir(path):
        if subject.is_file() and subject.name.endswith('.mat'):
            mat = sio.loadmat(subject.path)['eeg_data_wrt_task_rep_no_eog_256Hz_last_beep']
            for label in range(mat.shape[0]):
                for j in range(mat[label].shape[0]):
                    eeg.append(mat[label][j][:64,:])
                    labels.append(words[label + 1])
                    patient_id.append(int(re.search("[0-9]+", subject.name).group(0)))
                    name.append(f"patient_{patient_id[-1]}_word_{labels[-1]}_trial_{j+1}")
    return [eeg,labels,patient_id, name]

# return augmented_data with given window_size and stride
def augmented_data(path, words, window_size = 256, stride = 64):
    EEG, Labels, Patient_id, file_name = load_EEG(path, words)
    X = []
    Y = []
    id = []
    augmented_file_name = []
    for eeg, label, patient_id, name in zip(EEG, Labels, Patient_id, file_name):
        for start in range(0, eeg.shape[1] - window_size + 1, stride):
            X.append(eeg[:,start: start + window_size])
            Y.append(label)
            id.append(patient_id)
            augmented_file_name.append(f"{name}_{start//stride + 1}")

    return [X, Y, id, augmented_file_name]


# retrieves the MPC(Mean Phase Coherance) feature matrix for given EEG 64 channel
def MPC(eeg):
    channels = eeg.shape[0]
    mpc_matrix = np.zeros((channels, channels), dtype = float)

    def MPC_feature(i,j):
        signal_a = np.unwrap(np.angle(scipy.signal.hilbert(eeg[i])))
        signal_b = np.unwrap(np.angle(scipy.signal.hilbert(eeg[j])))
        phase_diff = np.exp((signal_a - signal_b) * 1j)
        return np.absolute(np.mean(phase_diff))
        
    for i in range(channels):
        for j in range(channels):
            if i <= j:
                mpc_matrix[i, j] = MPC_feature(i,j)
            else:
                mpc_matrix[i, j] = mpc_matrix[j, i]
    return mpc_matrix

    
# retrieves the MSC(Magnitude Phase Coherance) feature matrix for given EEG 64 channel
def MSC(eeg):
    channels = eeg.shape[0]
    msc_matrix = np.zeros((channels, channels), dtype = float)
        
    for i in range(channels):
        for j in range(channels):
            if i <= j:
                msc_matrix[i, j] = np.mean(scipy.signal.coherence(eeg[i], eeg[j], window = scipy.signal.windows.hamming(32), fs = 256)[1])
            else:
                msc_matrix[i, j] = msc_matrix[j, i]
    return msc_matrix


# alpha beta gamma filtering for every eeg electrode    
def alpha_beta_gamma_extractor(eeg):
    a = scipy.signal.butter(8, [8,13], 'bandpass', fs=256, output='sos')
    b = scipy.signal.butter(8, [13,30], 'bandpass', fs=256, output='sos')
    g = scipy.signal.butter(8, [30,70], 'bandpass', fs=256, output='sos')

    alpha = np.zeros_like(eeg)
    beta = np.zeros_like(eeg)
    gamma = np.zeros_like(eeg)

    for i in range(eeg.shape[0]):
        alpha[i] = scipy.signal.sosfilt(a, eeg[i])
        beta[i] = scipy.signal.sosfilt(b, eeg[i])
        gamma[i] = scipy.signal.sosfilt(g, eeg[i])
    
    return [alpha, beta, gamma]


# reutrn Image form of the eeg from alpha beta gamma bands and MPC and MSC feature matrix
def EEG_Image(eeg, **kwargs):
    eeg_channles = alpha_beta_gamma_extractor(eeg)
    Image = np.zeros((eeg.shape[0],eeg.shape[0],3), dtype=float)
    for i in range(3):
        eeg_mpc = MPC(eeg_channles[i])
        eeg_msc = MPC(eeg_channles[i])
        n = eeg_mpc.shape[0]
        for p in range(n):
            for q in range(n):
                if p < q:
                    Image[p,q,i] = eeg_mpc[p,q]
                elif p > q:
                    Image[p,q,i] = eeg_msc[p,q]
    return Image


# it extracts eeg Image and writes it into given location with proper name
def eeg_image_folder_maker(store_path, data, start, stop):
    X, Y, id, file_name = data
    for i in range(start,stop):
        with open(f"{store_path}/{file_name[i]}", 'wb') as f:
            np.save(f, EEG_Image(X[i]))
            np.save(f, Y[i])
            np.save(f, id[i])   


# extracts EEG image data from folder with augmentation method and writes to new location in parllel way
def parllel_feature_extracting(folder_name):
    data = augmented_data(folder_path[folder_name], Labels[folder_name], 256, 64)
    with concurrent.futures.ProcessPoolExecutor() as executor:
        N = len(data[1])
        cores = 40
        per_core = N // cores
        for i in range(0, N, per_core):
            executor.submit(eeg_image_folder_maker, eeg_image_folder_path[folder_name], data, i, i + per_core)


# returns particular patient's EEG image path and corresponding immagined word list
def Patient_id_data(eeg_image_path, patient_id):
    file_names = os.listdir(eeg_image_path)
    X = [f"{eeg_image_path}//{each}" for each in file_names if each.startswith(f"patient_{patient_id}")]
    return X

In [ ]:
class EEG_Dataset(Dataset):

    def __init__(self, eeg_image_path, patient_id, words):
        self.X = Patient_id_data(eeg_image_path, patient_id)
        self.words = words

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        with open(f"{self.X[idx]}", 'rb') as f:
            image = np.load(f).transpose(2,0,1)
            label = str(np.load(f))
        return torch.tensor(image, dtype= torch.float), self.words[label]

In [ ]:
dataset = EEG_Dataset(eeg_image_folder_path['Short_words'], 12, words['Short_words'])

dataloader = DataLoader(dataset, batch_size = 10, shuffle=True)

In [ ]:
# Helper function to show a batch
def show_landmarks_batch(sample_batched):
    images_batch, word_batch = sample_batched
    batch_size = len(images_batch)

    for i in range(batch_size):
        print(i, images_batch[i].shape,images_batch[i].dtype ,word_batch[i])

# if __name__ == '__main__':
for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched[0].size(),
          len(sample_batched[1]))

    # observe 4th batch and stop.
    if i_batch == 3:
        show_landmarks_batch(sample_batched)
        break

In [ ]:
train_dataset, val_dataset = random_split(dataset, [0.85,0.15], generator=torch.Generator().manual_seed(42))

In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation

data_dir = 'data/hymenoptera_data'
EEG_image_datasets = {'train': train_dataset, 'valid': val_dataset}
dataloaders = {x: DataLoader(EEG_image_datasets[x], batch_size=64,shuffle=True, num_workers=3) for x in ['train', 'valid']}
dataset_sizes = {x: len(EEG_image_datasets[x]) for x in ['train', 'valid']}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, number_of_words):
        super(NeuralNetwork, self).__init__()
        self.ResNet = models.resnet50(pretrained=True)
        # for param in self.ResNet.parameters():
        #     param.requires_grad = False
        self.fc1 = nn.Linear(self.ResNet.fc.in_features, 128)
        self.ResNet.fc = nn.Identity()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, number_of_words)
        

    def forward(self, x):
        x = self.ResNet(x)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = nn.functional.dropout(x, 0.3)
        x = self.fc2(x)
        x = nn.functional.relu(x)
        x = nn.functional.dropout(x, 0.3)
        x = self.fc3(x)
        x = nn.functional.softmax(x)
        return x

model = NeuralNetwork(3).to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size = 250, gamma=0.1)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        if not epoch % 50:
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                # print( loss.item(), outputs, labels,running_loss)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            if not epoch % 50:
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}\n')

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler,num_epochs = 300)

subject 2 long words Best val Acc: 0.613725

subject 3 long words Best val Acc: 0.633333

short long words subj 14 Best val Acc: 0.798039